## Ageing transcriptome meta-analysis reveals similarities and differences between key mammalian tissues, 2021, Aging, Palmer D. et al.<hr>

### Introduction
This notebook and project by Skoltech students tries to reproduce the results of the original paper studying transcriptomic changes associated with ageing in process.<br>For the sake of time we have focussed on mice only, while original paper studied and compared gene expression association with age for mice, rats and humans. For the analysis the ageing transcriptome datasets were used. These usually represent expreriments where several animals/individuals of different age are studied by RNA-sequencing techniques such as total RNA-seq or microarray studies.<br>For our analysis we used 27 datasets in total majority of which were microarray studies (used datasets as well as metafile can be found on our <a href="https://github.com/d-kozhevnikova/Ageing-transcriptome-meta-analysis">GitHub</a> reporsitory). The datasets were already provided by authors as normalized read counts for each gene (RPKM counts that are log2 transformed). We analyzed three tissues:
<ul>
<li>Heart tissue (5 datasets)</li>
<li>Brain tissue (12 datasets)</li>
<li>Muscle tissue (10 datasets)</li>
</ul>
In general the pipeline for the gene expression analysis looks as following:
<ol>
<li>Running linear regression analysis for each gene inside each dataset (age of samples is regressed against gene expression)</li>
<li>Selecting the genes that are:</li>
    <ul>
    <li>Statistically significantly assoicated with age (regression line slope)</li>
    <li>Are associated with age across multiple datasets (at least two)</li>
    </ul>
<li>Running meta-regression analysis:</li>
    <ul>
    <li>Using binomial cimulative test which requires:</li>
        <ol>
            <li>Prior calculation of probabilities for a random gene to be overexpressed and underexpressed - as average across multiple datastes of a percentage of differentially expressed genes from total number of studied genes</li>
            <li>Calculation of binomial probability where <i>N</i> is total number of datasets where gene is studies and <i>K</i> is the number of datasets where gene is statistically significantly over/underexpressed with age</li>
        </ol>
    <li>Using <a href="https://pymare.readthedocs.io/en/latest/generated/pymare.core.meta_regression.html#pymare.core.meta_regression" >PyMare</a> meta-regression function</li>
    </ul>    
</ol>
Based on this we obtained lists of genes that are differentially (over- and under-) expressed with age across multiple datasets for heart, brain, and muscle tissues individually as well as for combined analysis all together.
Further steps of the analysis involved:
<ul>
<li>Running the EnrichGO analysis for gene lists using <a href="https://gseapy.readthedocs.io/en/latest/introduction.html">gseapy</a> python package</li>
<li>Intersecting gene lists between different tissues</li>
<li>Intersecting gene lists with <a href="https://genomics.senescence.info/genes/index.html">GenAge</a> database</li> 
</ul>
<br>
Below is the code that has been used for the analysis of the datasets. All the packages required for the analysis are mentioned in the first cell of the code.<br>
In case of any questions please feel free to reach us out to us:
<ul>
<li>Telegram: @stappyman</li>
<li>E-mail: stapranad@mail.ru</li>
</ul><hr>

### Main code:<br>Preparing the data for the analysis
The cell below uploads packages required for data analysis and visualisation. During the analysis we have created a separate conda environment which made the analysis and package uploading easier and quicker. The needed packages can be download with following commands:
```Python
!pip install "package_name"
!conda install -c bioconda "package_name"
```

In [ ]:
import pandas as pd
import numpy as np
import scipy
import statsmodels.api as sm
import mpmath
import wrapt
import pymare
from pymare import core
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels
import mygene
import gseapy
from gseapy import barplot, dotplot
import openpyxl
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from rpy2 import robjects
from matplotlib_venn import venn3, venn2

### Selecting the datasets that are meeting our criteria:
The cell below allows selection of the datasets based on <i>metadata.xlsx</i> file. Datasets can be searched based on the following criteria:
<ul>
<li>Species (either human, mouse or rat)</li>
<li>Experimental technique (either RNAseq, Microarray or all)</li>
<li>Tissue (requires a list of tissues)</li>    
</ul>
It return the list of dataset names we need to use for the analysis

In [4]:
import pandas as pd
#reading the metadata column:
metadata = pd.read_excel('/Users/andrewstapran/Desktop/Skoltech/Term_8/Computational_Biology_of_Ageing/rnaseq-project-CBA/metadata.xlsx',sheet_name='TableS1')
metadata.columns = metadata.loc[0]
metadata = metadata[1:]

def input_list(df,Species,Technology,Tissue):
    subset1 = df[df['Species'] == Species]
    
    if Technology != 'all':
        if Technology == 'Microarray':
            subset2 = subset1[subset1['Technology'] == 'Microarray']
        elif Technology == 'RNAseq':
            subset2 = subset1[subset1['Technology'] == 'RNAseq']
    else:
        subset2 = subset1
    
    if Tissue != 'all':
        subset3 = subset2[subset2['Tissue'].isin(Tissue)]
    else:
        subset3 = subset2
        
    return list(subset3['Name_Dataset'])



#mylist = input_list(metadata,'Mouse','Microarray',['Brain','Heart','Muscle'])
mylist = input_list(metadata,'Mouse','all',['Muscle','Heart','Brain'])
mylist

['AgeMap.Brain.RegressionInput.txt',
 'AgeMap.Heart.RegressionInput.txt',
 'AgeMap.Muscle.RegressionInput.txt',
 'GDS1311.Brain.RegressionInput.txt',
 'GDS2082.Brain.RegressionInput.txt',
 'GDS2612.Muscle.RegressionInput.txt',
 'GDS2912.Brain.RegressionInput.txt',
 'GDS2972.Heart.RegressionInput.txt',
 'GDS2973.Brain.RegressionInput.txt',
 'GDS2996.Muscle.RegressionInput.txt',
 'GDS3620.Brain.RegressionInput.txt',
 'GDS40.Heart.RegressionInput.txt',
 'GDS4804.Muscle.RegressionInput.txt',
 'GDS4892.Muscle.RegressionInput.txt',
 'GDS4904.Muscle.RegressionInput.txt',
 'GSE11291.Brain.RegressionInput.txt',
 'GSE11291.Heart.RegressionInput.txt',
 'GSE11291.Muscle.RegressionInput.txt',
 'GSE12480.Heart.RegressionInput.txt',
 'GSE19676.Muscle.RegressionInput.txt',
 'GSE34378.Brain.RegressionInput.txt',
 'GSE48911.Brain.RegressionInput.txt',
 'GSE55163.Muscle.RegressionInput.txt',
 'GSE61915.Brain.RegressionInput.txt',
 'GSE69952.Brain.RegressionInput.txt',
 'GSE7958.Brain.RegressionInput.txt'

### Controlling for samples homogeneity using PCA analysis
This part of the code returns a PCA analysis plot based on expression of all genes present in the dataset for all samples. Initially the idea was to remove the sampples that fall outside their age cluster. However, it seemed that age groups were not as clearly separated on the PCA plot, so we just left all of them for the further analysis.<br>
In case you will be using this code please change the paths to reporsitory with gene count values which you can find in our <a href="https://github.com/d-kozhevnikova/Ageing-transcriptome-meta-analysis">GitHub</a> reporsitory. These are marked with the following comment:
```Python
### CHANGE THE PATH
```

In [ ]:
#quality control:
#function to read the input list of datasets
def dataset_reader(input_dataset):
    if 'AgeMap' in str(input_dataset):
        ### CHANGE THE PATH
        df_main = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{input_dataset}',sep=' ',skiprows=[0,1],header=None)
        ### CHANGE THE PATH
        df_header = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{input_dataset}',sep='\t',nrows=2,header=None)
        df = pd.concat([df_header,df_main]).reset_index(drop=True)
    else:
        ### CHANGE THE PATH
        df = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{input_dataset}',sep='\t',header=None)
    df.loc[0,0] = ''
    df.loc[1,0] = 'Age'
    df.columns = df.iloc[0]
    df = df[1:]
    df.set_index('',inplace=True)
    df_transformed = df.T
    return df

#plotting function for PCA analysis results
def plotterPCA(input_dataset):
    my_dataset = dataset_reader(input_dataset)
    meta = pd.DataFrame([list(my_dataset.columns),my_dataset.loc['Age']]).T
    meta.columns = ['','Age']
    meta.set_index('', inplace=True)
    meta['Age']

    x = np.transpose(my_dataset.values)
    x = StandardScaler().fit_transform(x)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])
    pcdf = pd.concat([principalDf.set_index(meta.index), meta], axis = 1)

    fig, ax = plt.subplots(figsize=(6, 6))
    sns.set_context("paper", font_scale=1.5)
    sns.scatterplot(data=pcdf, x="PC1", y="PC2", hue="Age", s = 50)
    plt.legend(loc="upper right", fontsize=10)

    var_pc1 = (pca.explained_variance_ratio_[0]*100).round(3)
    var_pc2 = (pca.explained_variance_ratio_[1]*100).round(3)
    plt.xlabel('PC1, ('+ var_pc1.astype(str) + "%)", fontsize = 20, labelpad = 10)
    plt.ylabel('PC2, ('+ var_pc2.astype(str) + "%)", fontsize = 20, labelpad = 2)
    plotname = input_dataset.replace('.RegressionInput.txt','').replace('.',' ')
    plt.title(plotname, fontsize = 20)
    ax.grid()

#running the function for one of the datasets in the mylist (list of dataset names that are currently being analysed)
plotterPCA(mylist[4])




### Running linear regression for each gene for each dataset
This cell runs the liniear regression for each gene using <a href="https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html">statsmodels.OLS</a> function. Age values of samples in the dataset are used as predictors (X) and gene expression values for a particular gene (Y) as response variables. <i>P-values</i> and <i>standard errors</i> are obtained from the output of the same function.<br>The following function are created here:
<ol>
<li><b><i>return_estimators</i></b></li>
    <ul>
        <li><b>Takes in</b> the list of dataset names obtained in the previous section code</li>
        <li>Reads the initial tsv file into pandas dataframe</li>
        <li>Runs the linear regression for each gene with the dataset</li>
        <li><b>Returns</b> the list of following parameters:</li>
            <ol>
                <li>Genes differentially expressed with age in this dataset</li>
                <li>Coefficients for regression slopes for these genes</li>
                <li>Standard errors for these coefficients</li>
                <li>Datasets studied</li>
                <li>Raw expression values for each gene for each dataset</li>
                <li>Number of samples in each dataset</li>
                <li>All genes studied in these datasets</li>
                <li>List of % of underexpressed genes from total gene number for each dataset</li>
                <li>List of % of overexpressed genes from total gene number for each dataset</li>
            </ol>
    </ul><br>
<li><b><i>return_estimators_binomial</i></b></li>
    <ul>
        <li>Does similar things to what the <b><i>return_estimators</i></b> is doing, however selects only genes with statistically significant slopes for the binomial cumulative analysis</li>
        <li><b>Returns</b> the list of following parameters:</li>
            <ol>
                <li>Genes differentially expressed with age in this dataset</li>
                <li>Coefficients for regression slopes for these genes</li>
                <li>Standard errors for these coefficients</li>
                <li>Datasets studied</li>
                <li>Raw expression values for each gene for each dataset</li>
                <li>Number of samples in each dataset</li>
            </ol>
    </ul><br>
<li><b><i>dataset_builder</i></b></li>
    <ul>
        <li><b>Takes in</b> the parameters returned by the previous two functions and constructs the datasets with columns as <i>datasets</i> and rows as <i>genes</i> - in the intersections either coefficients, standard errors or something else is given</li>
        <li><b>Returns</b> the list of following parameters:</li>
            <ol>
                <li>Dataframe for regression slopes coefficients</li>
                <li>Dataframe for standard errors for regression slopes coefficients</li>
                <li>Dataframe for expression count values for each gene for each dataset</li>
                <li>Sample size for each dataset</li>
            </ol>
    </ul><br>
<li><b><i>binomial_estimator</i></b></li>
    <ul>
        <li><b>Takes in</b> the parameters returned by the <b><i>return_estimators_binomial</i></b> and <b><i>return_estimators</i></b> functions + writes union of all genes studied in selected datasets to specified file</li>
        <li><b>Returns</b> the list of following parameters:</li>
            <ol>
                <li>Dictionary with genes as keys and total occurences number (in how many datasets they are studied) </li>
                <li>Average percentage of statistically significantly underexpressed genes (from total number of genes studied)</li>
                <li>Average percentage of statistically significantly overexpressed genes (from total number of genes studied)</li>
            </ol>
    </ul><br>
</ol>

In [ ]:
#This function reads the initial tsv file into pandas dataframe
#After this it runs the linear regression for each gene with the dataset
def return_estimators(list_input):
    datasets = []
    genes = []
    coefs = []
    std_errs = []
    dataset_values = []
    sample_number_list = []
    all_genes_studied = []
    probabilities_under = []
    probabilities_over = []
    
    for each in list_input:
        print(each)
        datasets.append(each)
        if 'AgeMap' in str(each):
            ### CHANGE THE PATH
            df_main = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep=' ',skiprows=[0,1],header=None)
            ### CHANGE THE PATH
            df_header = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep='\t',nrows=2,header=None)
            df = pd.concat([df_header,df_main]).reset_index(drop=True)
        else:
            ### CHANGE THE PATH
            df = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep='\t',header=None)
        df.loc[0,0] = ''
        df.loc[1,0] = 'Age'
        df.columns = df.iloc[0]
        df = df[1:]
        df.set_index('',inplace=True)
        sample_number = len(list(df.columns))
        sample_number_list.append(sample_number)
        raw_values = ['age'] + list(df.values.tolist()[1:])
        df_transformed = df.T
        x = np.array(list(df_transformed['Age'].astype(float)))#.reshape((-1, 1))
        list_slopes = ['age']
        p_values = ['age']
        std_errs_list = ['age']
        
        for each in list(df_transformed.columns)[1::]:
            y = np.array(list(df_transformed[each].astype(float)))
            X = sm.add_constant(x)
            model = sm.OLS(y,X).fit()
            p_value = model.pvalues[-1]
            std_err = model.bse[-1]
            coef = model.params[-1]
            
            list_slopes.append(coef)
            p_values.append(p_value)
            std_errs_list.append(std_err)
            
        df_transformed.loc['slopes'] = list_slopes
        df_transformed.loc['pvalues'] = p_values
        df_transformed.loc['std_errs'] = std_errs_list
        df_transformed.loc['values'] = raw_values
        
        df_final = df_transformed.T
        all_genes_studied.append(list(df_final.index)[1:])
        df_final2 = df_final[df_final['pvalues'] != 'age']
        df_for_binomial = df_final2[df_final2['pvalues'] < 0.05]
        df_final_stat_sign = df_final2
        
        genes.append(list(df_final_stat_sign.index))
        coefs.append(list(df_final_stat_sign['slopes']))
        std_errs.append(list(df_final_stat_sign['std_errs']))
        dataset_values.append(list(df_final_stat_sign['values']))
        
        total_genes_number = len(list(df_final.index)[1:])
        underexp_number = len(df_for_binomial[df_for_binomial['slopes'] < 0])
        print(f'Total number of genes underexpressed with age: {underexp_number}')
        overexp_number = len(df_for_binomial[df_for_binomial['slopes'] > 0])
        print(f'Total number of genes overexpressed with age: {overexp_number}')
        probabilities_under.append(float(underexp_number / total_genes_number))
        probabilities_over.append(float(overexp_number / total_genes_number))
        print('---------------')
        
    
    #dataset = dataset_builder([genes,coefs,datasets])
    return [genes,coefs,std_errs,datasets,dataset_values,
            sample_number_list,all_genes_studied,
            probabilities_under,probabilities_over,]







def return_estimators_binomial(list_input):
    datasets = []
    genes = []
    coefs = []
    std_errs = []
    dataset_values = []
    sample_number_list = []
    all_genes_studied = []
    probabilities_under = []
    probabilities_over = []
    
    for each in list_input:
        print(each)
        datasets.append(each)
        if 'AgeMap' in str(each):
            ### CHANGE THE PATH
            df_main = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep=' ',skiprows=[0,1],header=None)
            ### CHANGE THE PATH
            df_header = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep='\t',nrows=2,header=None)
            df = pd.concat([df_header,df_main]).reset_index(drop=True)
        else:
            ### CHANGE THE PATH
            df = pd.read_csv(f'/Users/andrewstapran/Desktop/rnaseq-project-CBA/AgeingSignatures2020_supplementary/SupplementaryDatasets1-RegressionInputs/{each}',sep='\t',header=None)
        df.loc[0,0] = ''
        df.loc[1,0] = 'Age'
        df.columns = df.iloc[0]
        df = df[1:]
        df.set_index('',inplace=True)
        sample_number = len(list(df.columns))
        sample_number_list.append(sample_number)
        raw_values = ['age'] + list(df.values.tolist()[1:])
        df_transformed = df.T
        x = np.array(list(df_transformed['Age'].astype(float)))#.reshape((-1, 1))
        list_slopes = ['age']
        p_values = ['age']
        std_errs_list = ['age']
        
        for each in list(df_transformed.columns)[1::]:
            y = np.array(list(df_transformed[each].astype(float)))
            X = sm.add_constant(x)
            model = sm.OLS(y,X).fit()
            p_value = model.pvalues[-1]
            std_err = model.bse[-1]
            coef = model.params[-1]
            
            list_slopes.append(coef)
            p_values.append(p_value)
            std_errs_list.append(std_err)
            
        df_transformed.loc['slopes'] = list_slopes
        df_transformed.loc['pvalues'] = p_values
        df_transformed.loc['std_errs'] = std_errs_list
        df_transformed.loc['values'] = raw_values
        
        df_final = df_transformed.T
        all_genes_studied.append(list(df_final.index)[1:])
        df_final2 = df_final[df_final['pvalues'] != 'age']
        df_final_stat_sign = df_final2[df_final2['pvalues'] < 0.05]
        
        genes.append(list(df_final_stat_sign.index))
        coefs.append(list(df_final_stat_sign['slopes']))
        std_errs.append(list(df_final_stat_sign['std_errs']))
        dataset_values.append(list(df_final_stat_sign['values']))
        
        total_genes_number = len(list(df_final.index)[1:])
        underexp_number = len(df_final_stat_sign[df_final_stat_sign['slopes'] < 0])
        print(f'Total number of genes underexpressed with age: {underexp_number}')
        overexp_number = len(df_final_stat_sign[df_final_stat_sign['slopes'] > 0])
        print(f'Total number of genes overexpressed with age: {overexp_number}')
        probabilities_under.append(float(underexp_number / total_genes_number))
        probabilities_over.append(float(overexp_number / total_genes_number))
        print('---------------')
        
    
    #dataset = dataset_builder([genes,coefs,datasets])
    return [genes,coefs,std_errs,datasets,dataset_values,
            sample_number_list]






#combining linear regression coefficients, standard errors from different datasets into one dataframe
def dataset_builder(input_coefs_genes):
    input_genes = input_coefs_genes[0]
    input_coefs = input_coefs_genes[1]
    input_stderrs = input_coefs_genes[2]
    input_datasets = input_coefs_genes[3]
    input_values = input_coefs_genes[4]
    input_sample_size = input_coefs_genes[5]
    
    dataset_name = input_datasets[0].replace('.RegressionInput.txt','')
    initial_coefs = pd.DataFrame([input_genes[0],input_coefs[0]]).T
    initial_stderrs = pd.DataFrame([input_genes[0],input_stderrs[0]]).T
    initial_values = pd.DataFrame([input_genes[0],input_values[0]]).T
    
    initial_coefs.columns = ['Gene',dataset_name]
    initial_stderrs.columns = ['Gene',dataset_name]
    initial_values.columns = ['Gene',dataset_name]
    
    initial_coefs['Gene'] = initial_coefs['Gene'].astype(int)
    initial_stderrs['Gene'] = initial_stderrs['Gene'].astype(int)
    initial_values['Gene'] = initial_values['Gene'].astype(int)
    
    for index, element in enumerate(input_genes[1:]):
        dataset_name = input_datasets[index+1].replace('.RegressionInput.txt','')
        to_add_coefs = pd.DataFrame([element,input_coefs[index+1]]).T
        to_add_stderrs = pd.DataFrame([element,input_stderrs[index+1]]).T
        to_add_values = pd.DataFrame([element,input_values[index+1]]).T
        
        to_add_coefs.columns = ['Gene',dataset_name]
        to_add_stderrs.columns = ['Gene',dataset_name]
        to_add_values.columns = ['Gene',dataset_name]
        
        to_add_coefs['Gene'] = to_add_coefs['Gene'].astype(int)
        to_add_stderrs['Gene'] = to_add_stderrs['Gene'].astype(int)
        to_add_values['Gene'] = to_add_values['Gene'].astype(int)
        
        initial_coefs = pd.merge(initial_coefs,to_add_coefs,on='Gene',how='outer').fillna('not_pres')
        initial_stderrs = pd.merge(initial_stderrs,to_add_stderrs,on='Gene',how='outer').fillna('not_pres')
        initial_values = pd.merge(initial_values,to_add_values,on='Gene',how='outer').fillna('not_pres')
        
    initial_coefs.set_index('Gene',inplace=True)
    initial_stderrs.set_index('Gene',inplace=True)
    initial_values.set_index('Gene',inplace=True)
    return [initial_coefs,initial_stderrs,initial_values,input_sample_size]
        





def binomial_estimator(input_result,total_studied_path):
    list_all_genes = input_result[6]
    final_list = []
    for element in list_all_genes:
        final_list = final_list + [str(i) for i in element]
    print(f'Total number of genes: {len(final_list)}')
    final_list_unique = list(set(final_list))
    total_genes = open(total_studied_path,'w')
    for gene_element in final_list_unique:
        total_genes.write(f'{gene_element}\n')
    total_genes.close()
    print(f'Total number of unique genes: {len(final_list_unique)}')
    gene_occurences = {}
    for element in final_list_unique:
        print(f'{final_list_unique.index(element)} out of {len(final_list_unique)}')
        counter = 0
        for each in list_all_genes:
            new_list = [str(bublik) for bublik in each]
            if element in new_list:
                counter += 1
        gene_occurences[element] = counter
    
    average_under_P = sum(input_result[7]) / len(input_result[7])
    average_over_P = sum(input_result[8]) / len(input_result[8])
    return [gene_occurences, average_under_P, average_over_P]



#running the functions
a = return_estimators(mylist)        
mydata = dataset_builder(a)
### CHANGE THE PATH
general_stats_on_selected_datasets = binomial_estimator(a,'/Users/andrewstapran/Desktop/mygenesall.txt')



### Subsetting the dataframe for datasets
In the previous step we have obtained the dataset with expression-age regression slope coefficients for each gene for each dataset. Now we need to select genes that are associated with age in <b><i>at least 2 datasets</i></b>.<br>

In [ ]:
def subsetting_stat_genes(df):
    #finding genes that were studied in at least 2 datasets
    dataset_number = len(df[0].columns)
    # if dataset_number % 2 == 0:
    #     threshold = dataset_number / 2
    # else:
    #     threshold = int(dataset_number / 2) + 1

    coefs = df[0]
    stderrs = df[1]
    values = df[2]
    sample_size = df[3]

    genes_studied = list(coefs.index)
    genes_to_study = []
    for element in genes_studied:
        number_abs = list(coefs.loc[element]).count('not_pres')
        if int(number_abs) <= dataset_number - 2:
            genes_to_study.append(element)
        else:
            pass

    coefs_atleast = coefs.loc[genes_to_study]    
    stderrs_atleast = stderrs.loc[genes_to_study]
    values_atleast = values.loc[genes_to_study]
    return [coefs_atleast,stderrs_atleast,values_atleast,sample_size]

myraw = subsetting_stat_genes(mydata)
myraw_binomial = subsetting_stat_genes(mydata_binomial)

<hr>

### Main code:<br>Running <b><i>metaregression</i></b> analysis
There are two function below for running the meta-regression analysis:
<ol>
    <li><i><b>gene_list_return</b></i> using PyMare meta-regression</li>
    <li><i><b>binomial_check</b></i> using Binomial Cumulative test meta-regression</li>
</ol>
Both tests use multiple test correction for p-value adjustment using the <a href="https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.fdrcorrection.html">statsmodels.stats.multitest.fdrcorrection</a> function

In [ ]:
def not_pres_remove(input_list):
    final_list = [element for element in input_list if element != 'not_pres']
    return final_list

enc = preprocessing.OneHotEncoder()
a = np.array(['Heart','Muscle','Heart','Brain','Heart']).reshape(-1, 1)
b = enc.fit(a)

def gene_list_return(input_data):
    coefs_pymare = input_data[0].copy()
    column_names = [each.split('.')[-1] for each in list(coefs_pymare.columns)]
    column_names_corr = enc.transform(np.array(column_names).reshape(-1, 1)).toarray()
    
    variances = input_data[1].copy()
    dataset_size = input_data[3].copy()
    p_values = []
    coefs_final = []
    for element in list(coefs_pymare.index):
        print(f'{element} --> {list(coefs_pymare.index).index(element)} out of {len(list(coefs_pymare.index))}')
        initial_elements = list(coefs_pymare.loc[element])
        y = not_pres_remove(list(coefs_pymare.loc[element]))
        y_num = [float(each) for each in y]
        v = not_pres_remove(list(variances.loc[element]))
        v_num = [float(each) for each in v]
        v_squared = [i ** 2 for i in v_num]
        omit_list = [index for index, element in enumerate(initial_elements) if element == 'not_pres']
        study_list_size = [element for index, element in enumerate(dataset_size) if index not in omit_list]
        column_names_list = [each for index1, each in enumerate(column_names_corr) if index1 not in omit_list]
        model = core.meta_regression(y = y_num, v = v_squared, X = column_names_list)
        #model = core.meta_regression(y = y_num, v = v_squared, n = study_list_size)
        final_df = model.to_df(alpha=0.05)
        p_value = final_df.loc[0,'p-value']
        estimate = final_df.loc[0,'estimate']
        p_values.append(p_value)
        coefs_final.append(estimate)
    coefs_pymare['total_effect'] = coefs_final
    coefs_pymare['total_p_values'] = p_values
    coefs_pymare_subset = coefs_pymare[coefs_pymare['total_p_values'].notna()]
    coefs_pymare_subset['adjusted_p_values'] = list(statsmodels.stats.multitest.fdrcorrection(list(coefs_pymare_subset['total_p_values']),alpha=0.05,is_sorted=False)[1])
    #coefs_pymare['adjusted_p_values'] = list(statsmodels.stats.multitest.fdrcorrection(p_values,alpha=0.05,is_sorted=False)[1])
    return coefs_pymare_subset



#binomial test metaregression:
def binomial_check(input_data, general_stats_input):
    coefs_binom2 = input_data[0].copy()
    genes_left = [int(gene) for gene in list(final_meta_df.index)]
    intersection = [int(gene_ID) for gene_ID in genes_left if int(gene_ID) in list(coefs_binom2.index)]
    coefs_binom = coefs_binom2.loc[intersection]
    print(len(coefs_binom))
    datasets = list(coefs_binom.columns)
    print(datasets)
    total_occurences = general_stats_input[0]
    under_P_average = general_stats_input[1]
    over_P_average = general_stats_input[2]
    binomial_Ps = []
    for each in list(coefs_binom.index):
        list_row = [float(element) for element in list(coefs_binom.loc[each]) if element != 'not_pres']
        k_pos = len([item for item in list_row if float(item) > 0])
        k_neg = len([item for item in list_row if float(item) < 0])
        if k_pos > k_neg:
            probability = over_P_average
            k = k_pos
        elif k_pos < k_neg:
            probability = under_P_average
            k = k_neg
        else:
            k = k_pos
            effect = final_meta_df.loc[each,'total_effect']
            if effect > 0:
                probability = over_P_average
            elif effect < 0:
                probability = under_P_average
        n = total_occurences[str(each)]
        binomial_Ps.append(1 - scipy.stats.binom.cdf(k, n, probability, loc=0))
    final_df = pd.DataFrame([[str(each) for each in list(coefs_binom.index)],binomial_Ps]).T
    final_df.columns = ['Genes','p_value']
    final_df.set_index('Genes', inplace=True)
    p_values = [float(element) for element in list(final_df['p_value'])]
    final_df['adjusted_p_value'] = list(statsmodels.stats.multitest.fdrcorrection(p_values,alpha=0.05,is_sorted=False)[1])
    return final_df

#running the meta-regression analysis
final_meta_df = gene_list_return(myraw)
binomial_results = binomial_check(myraw_binomial, general_stats_on_selected_datasets)
subset_final_pymare = final_meta_df.loc[[int(each) for each in list(binomial_results.index)]]
binomial_results['effect_meta'] = list(subset_final_pymare['total_effect'])


        
        

### Comparing the results of PyMare and Binomial meta-regression:
This part of the code plots the results of PyMare meta-regression against Binomial meta-regression. Each approach gives a specific p-value to each gene based on how frequently stastically signifincant association of this gene is observed in the studied datasets. Therefore we can rank these genes from based on their p-values (starting with best up-regulated genes and ending with best down-regulated genes). And then plot the rank positions of the genes commonly identified by both analysis types, calculating the correlaiton coefficient. This is returned as output of the function

In [ ]:
def group_pos(input_df,typeofdata):
    positions = list(range(0,len(input_df),1))
    input_df['index_pos'] = positions
    if typeofdata == 'binomial':
        input_df_grouped = input_df.groupby('adjusted_p_value')['index_pos'].mean().reset_index()
        input_df_dict = dict(zip(list(input_df_grouped.adjusted_p_value),list(input_df_grouped.index_pos)))
        final_pos = dict(zip([int(each) for each in list(input_df.index)],[input_df_dict[each] for each in list(input_df.adjusted_p_value)]))
    elif typeofdata == 'pymare':
        input_df_grouped = input_df.groupby('adjusted_p_values')['index_pos'].mean().reset_index()
        input_df_dict = dict(zip(list(input_df_grouped.adjusted_p_values),list(input_df_grouped.index_pos)))
        final_pos = dict(zip([int(each) for each in list(input_df.index)],[int(input_df_dict[each]) for each in list(input_df.adjusted_p_values)]))
    return final_pos

def plotter(binomial_input,pymare_input):
    binomial_input_nonzero = binomial_input[binomial_input['adjusted_p_value'] != 0]
    list_genes_binomial = [int(each) for each in list(binomial_input_nonzero.index)]
    pymare_input_nonzero = pymare_input[pymare_input['adjusted_p_values'] != 0]
    list_genes_pymare = [int(each) for each in list(pymare_input_nonzero.index)]
    
    common_list_binomial = [str(each) for each in list_genes_binomial if each in list_genes_pymare]
    common_list_pymare = [int(each) for each in list_genes_binomial if each in list_genes_pymare]
    binomial_subset = binomial_input_nonzero.loc[common_list_binomial]
    pymare_subset = pymare_input_nonzero.loc[common_list_pymare]
    
    pymare_over_subset = pymare_subset[pymare_subset['total_effect'] > 0]
    pymare_over_subset.sort_values(by=['adjusted_p_values'],ascending=True,inplace=True)
    pymare_under_subset = pymare_subset[pymare_subset['total_effect'] < 0]
    pymare_under_subset.sort_values(by=['adjusted_p_values'],ascending=False,inplace=True)
    binomial_over_subset = binomial_subset[binomial_subset['effect_meta'] > 0]
    binomial_over_subset.sort_values(by=['adjusted_p_value'],ascending=True,inplace=True)
    binomial_under_subset = binomial_subset[binomial_subset['effect_meta'] < 0]
    binomial_under_subset.sort_values(by=['adjusted_p_value'],ascending=False,inplace=True)
    pymare_final = pd.concat([pymare_over_subset,pymare_under_subset])
    binomial_final = pd.concat([binomial_over_subset,binomial_under_subset])
    #pymare_final = pymare_under_subset
    #binomial_final = binomial_under_subset
    
    list_pymare = group_pos(pymare_final,'pymare')
    list_binomial = group_pos(binomial_final,'binomial')
#     list_pymare = [str(each) for each in list(pymare_final.index)]
#     list_binomial = [str(each) for each in list(binomial_final.index)]
    resulting_dict = {}
    for element in common_list_pymare:
        #element = str(element2)
        index_pymare = list_pymare[element]
        #index_pymare = list_pymare.index(element)
        index_binomial = list_binomial[element]
        #index_binomial = list_binomial.index(element)
        resulting_dict[index_pymare] = index_binomial
    plt.scatter(list(resulting_dict.keys()),list(resulting_dict.values()))
    plt.xlabel('PyMare analysis based rank', fontsize=15)
    plt.ylabel('Cumulative binomial test\nbased rank', fontsize=15)
    res = scipy.stats.spearmanr(list(resulting_dict.keys()), list(resulting_dict.values()), axis=0,alternative='two-sided')
    print(res)
    plt.text(max(list(resulting_dict.keys()))*0.5,max(list(resulting_dict.values()))*0.1,f'Spearman:\nCorrelation: {res.correlation:.4f}\nP-value: {res.pvalue:.4f}',backgroundcolor = 'lightgray')    
    #plt.text(5000,5000,res.correlation,backgroundcolor = 'gray')    
    plt.show()
    return [resulting_dict,pymare_final,binomial_final]

a = plotter(binomial_results,final_meta_df)
    

### Converting Entrez_ID to Gene_IDs
Original datasets have been using the Entrez_ID identifiers for the genes which look something like this: <b><i>103018</i></b>. However using <a href="https://pypi.org/project/mygene/">mygene</a> package we converted them to conventional gene names, like <b><i>APOE</i></b>. Afterwards, the resulting dataframe containing:
<ul>
    <li>Coefficients for each gene for each dataset</li>
    <li>EntrezID</li>
    <li>Conventional gene names</li>
    <li>Total effect column - output of the PyMare meta-regression function (somthing like an "average" regression coefficient for all datasets for this gene). If positive - upregulated with age, if negative - vice versa</li>
    <li>Raw p-values for total effect estimates</li>
    <li>Benjamini-Hochberg FDR adjusted p-values for total effect estimates</li>
</ul>
Importantly, some Entrez_ID correspond to genes that have either been discontinued from using in annotation or to some putative protein-coding regions. For such Entrez_ID mygene package is unable to find the conventional gene name - these are replaced with the <b><i>geneID_not_found</i></b> label. It is important to weed them out from gene lists before running the further analysis steps

In [ ]:
mg = mygene.MyGeneInfo()
identificator = 'threetissues_Xconsidering'
def list_gene_name_converter(input_gene_list):
    list_entrez_gene_IDs = [int(each) for each in input_gene_list]
    output = mg.getgenes(list_entrez_gene_IDs,fields='symbol')
    list_geneIDs = []
    for element in output:
        if 'symbol' in list(element.keys()):
            list_geneIDs.append(element['symbol'].upper())
        else:
            list_geneIDs.append('geneID_not_found')
    return list_geneIDs

def convert_gene_names(input_df):
    list_gene_IDs = list_gene_name_converter(list(input_df.index))
    return_df = input_df.copy()
    return_df['gene_name'] = list_gene_IDs
    return_df['mouse_entrez_id'] = list(return_df.index)
    cols = return_df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    return_df = return_df[cols] 
    return return_df

#running the function for PyMare results
meta_pymare = convert_gene_names(final_meta_df)
### CHANGE THE PATH
meta_pymare.to_csv(f'/Users/andrewstapran/Desktop/{identificator}_pymare.csv',sep='\t',header=True,index=False)

#similarly we can obtain similar table with binomial cumulative testing and run table creation for these
meta_binomial = convert_gene_names(final_df)
### CHANGE THE PATH
meta_binomial.to_csv(f'/Users/andrewstapran/Desktop/{identificator}_binomial.csv',sep='\t',header=True,index=False)



### Converting gene list to names of genes
[Section1](#myID1)
As you might remember we also saved list of all genes studied across selected datasets - however they are given as Entrez_ID numbers as well. This function converts gene numbers to conventional gene names

In [ ]:
mg = mygene.MyGeneInfo()
def list_gene_name_converter(input_gene_list):
    list_entrez_gene_IDs = [int(each) for each in input_gene_list]
    output = mg.getgenes(list_entrez_gene_IDs,fields='symbol')
    list_geneIDs = []
    for element in output:
        if 'symbol' in list(element.keys()):
            list_geneIDs.append(element['symbol'].upper())
        else:
            gene_ID_number = element['query']
            list_geneIDs.append(f'{gene_ID_number} - geneID_not_found')
    return list_geneIDs

list1 = []
### CHANGE THE PATH
with open('/Users/andrewstapran/Desktop/mygenesall.txt','r') as file1:
    lines = file1.readlines()
for element in lines:
    list1.append(element.replace('\n',''))
file1.close()
list2 = list_gene_name_converter(list1)

### CHANGE THE PATH
file2 = open('/Users/andrewstapran/Desktop/all_three_tissues_names.txt','w')
for element in list2:
    file2.write(f'{element}\n')
file2.close()

<hr>

### Main code:<br>Running the enrichGO analysis
Using this bit of code we ran the enrichment analysis for <a href="http://geneontology.org/">GO (Gene Ontology)</a> terms for up- and downregulated genes for each tissue as well as for meta-analysis of three tissues together. As input for the <b><i>gseapy.enrichr</i></b> function following lists are used:
<ol>
    <li>Gene list of statistically significantly up- or downregulated with age genes (we used <i>Gene</i> column from <i>csv</i> tables we have obtained in the previous step)</li>
    <li>As background genes: list of genes studied across all studied datasets in this analysis which can be obtained from the previous step <i>txt</i> tables we have obtained in the previous step</li>
</ol>

In [ ]:
### CHANGE THE PATH
brain_table = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/brain/brain_pymare.csv'
brain_file = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/brain/brain_all_genes_names.txt'
heart_table = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/heart/heart_pymare.csv'
heart_file = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/heart/heart_all_genes_names.txt'
muscle_table = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/muscle/muscle_pymare.csv'
muscle_file = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/muscle/muscle_all_genes_names.txt'
total_table = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/allthree/threetissues_Xconsidering_pymare.csv'
total_file = '/Users/andrewstapran/Desktop/rnaseq-project-CBA/results/allthree/all_three_tissues_names.txt'

#reading the dataframes
def reader(input_df_path,input_file_path):
    input_df = pd.read_csv(input_df_path,sep='\t')
    with open(input_file_path,'r') as file1:
        lines = file1.readlines()
    list_all_genes = [each.replace('\n','') for each in lines if 'not_found' not in each]
    stat_sign = input_df[input_df.adjusted_p_values < 0.05]
    stat_sign_over = stat_sign[stat_sign['total_effect'] > 0]
    stat_sign_under = stat_sign[stat_sign['total_effect'] < 0]
    stat_sign_over_final = [each for each in list(stat_sign_over['gene_name']) if 'not_found' not in each]
    stat_sign_under_final = [each for each in list(stat_sign_under['gene_name']) if 'not_found' not in each]
    return [stat_sign_over_final,stat_sign_under_final,list_all_genes]


#creating the lists of genes for each tissue
brain_over = reader(brain_table,brain_file)[0]
print(len(brain_over))
brain_under = reader(brain_table,brain_file)[1]
print(len(brain_under))
brain_total = reader(brain_table,brain_file)[2]
print(len(brain_total))
heart_over = reader(heart_table,heart_file)[0]
print(len(heart_over))
heart_under = reader(heart_table,heart_file)[1]
print(len(heart_under))
heart_total = reader(heart_table,heart_file)[2]
muscle_over = reader(muscle_table,muscle_file)[0]
muscle_under = reader(muscle_table,muscle_file)[1]
muscle_total = reader(muscle_table,muscle_file)[2]
three_over = reader(total_table,total_file)[0]
three_under = reader(total_table,total_file)[1]
three_total = reader(total_table,total_file)[2]

#part for the GO
def plot_GO(inputlist_diff,inputlist_total,title):
    enr = gseapy.enrichr(gene_list = inputlist_diff,
                         background = inputlist_total,
                         gene_sets=['GO_Biological_Process_2021', 
                                    'GO_Cellular_Component_2021', 
                                    'GO_Molecular_Function_2021'],
                         organism='mouse',
                         outdir=None)
    enr_res = enr.results.iloc[:,[0,1,2,4,9]]
    enr_res = enr_res[enr_res['Adjusted P-value'] < 0.05]
    enr_res
    dotplot(enr_res,
            column="Adjusted P-value",
            x='Gene_set',
            size=10,
            top_term=5,
            figsize=(3,5),
            title = title,
            thresh=0.5,
            xticklabels_rot=45, 
            show_ring=True, 
            marker='o')

#running the plotting functions    
plot_GO(brain_over,brain_total,'OverExpressed in brain')
plot_GO(brain_under,brain_total,'UnderExpressed in brain') 
plot_GO(muscle_over,muscle_total,'OverExpressed in muscle')
plot_GO(muscle_under,muscle_total,'UnderExpressed in muscle')     
plot_GO(heart_over,heart_total,'OverExpressed in heart')
plot_GO(heart_under,heart_total,'UnderExpressed in heart')    
plot_GO(three_over,three_total,'OverExpressed in All three tissues')
plot_GO(three_under,three_total,'UnderExpressed in All three tissues')   



### Building the venn diagrams for gene list intersections
This final section of the code intersects gene lists from different tissues with each other and builds venn diagrams for these intersections.<br>It also builds an intersection for genes up- and downregulated with age identified in meta-analysis of three tissues with GenAge gene list for mice

In [ ]:
def intersector(list1,list2):
    list3 = [each for each in list1 if each in list2]
    return list3

#OVER:
brain_heart_over = intersector(brain_over,heart_over)
brain_muscle_over = intersector(brain_over,muscle_over)
heart_muscle_over = intersector(muscle_over,heart_over)
all_three_over = intersector(brain_heart_over,muscle_over)
final_intersect = intersector(all_three_over, three_over)

#UNDER:
brain_heart_under = intersector(brain_under,heart_under)
brain_muscle_under = intersector(brain_under,muscle_under)
heart_muscle_under = intersector(muscle_under,heart_under)
all_three_under = intersector(brain_heart_under,muscle_under)
final_intersect = intersector(all_three_under, three_under)

### CHANGE THE PATH to GenAge table
GenAge = pd.read_csv('/Users/andrewstapran/Downloads/genage_models_export.tsv',sep='\t')
GenAge_genes_list = [element.upper() for element in list(GenAge['Gene Symbol'])]
GenAge_genes_list_unique = list(set(GenAge_genes_list))

#UNDER:
GenAge_heart_under = intersector(heart_under,GenAge_genes_list_unique)
GenAge_muscle_under = intersector(muscle_under,GenAge_genes_list_unique)
GenAge_brain_under = intersector(brain_under,GenAge_genes_list_unique)

#OVER:
GenAge_heart_over = intersector(heart_over,GenAge_genes_list_unique)
GenAge_muscle_over = intersector(muscle_over,GenAge_genes_list_unique)
GenAge_brain_over = intersector(brain_over,GenAge_genes_list_unique)

GenAge_total_over = intersector(three_over,GenAge_genes_list_unique)
GenAge_total_under = intersector(three_under,GenAge_genes_list_unique)



#function to build venn diagrams for intersection of three gene lists
def plot_venn3(list1,list2,list3, name1, name2, name3):
    venn3([set(list1), set(list2), set(list3)], 
          (name1, name2, name3))
    plt.show()
    

    
#function to build venn diagrams for intersection of three gene lists
def plot_venn2(list1):
    venn2([set(list1), set(GenAge_genes_list_unique)], 
          ('Three tissues overexpressed', 'GenAge'))
    plt.show()

<hr>

### Conclusion
With this code we were able to partially reproduce the results of original study. In particular:
<ol>
    <li>Immune function and inflammation are common GO features among gene up-regulated with age, especially in brain. This is consistent with inflammageing theory of ageing process</li>
    <li>In muscle genes important for cell differentiation were updegulated. These cahnges can be associated withh deterioration of muscle tissue regeneration with age</li>
    <li>Finally, metabolic genes were downregulated in heart muscle with age</li>
</ol>
Other results can be found in our <a href="https://github.com/d-kozhevnikova/Ageing-transcriptome-meta-analysis">GitHub</a> repository.
<br>
<br>
Credits for compiling the notebook and running the meta-regression analysis: <i>Andrei Stapran</i><br>
<i>Moscow, March, 2021</i>